In [ ]:
import os
import pydicom
from pydicom.misc import is_dicom
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import plotly.graph_objects as go
import nibabel as nib
import plotly.io as pio

In [ ]:
def load_dicom_images(directory, max_slices=None):
    slices = []
    for filename in sorted(os.listdir(directory)):
        if filename.endswith(".dcm"):
            path = os.path.join(directory, filename)
            dicom = pydicom.dcmread(path)
            slices.append(dicom)
            if max_slices and len(slices) >= max_slices:
                break
    # Ordenar as fatias por posição do paciente
    slices.sort(key=lambda x: x.ImagePositionPatient[2])
    return slices

def create_3d_image(slices):
    # Extrair arrays de pixel e empilhá-los
    pixel_arrays = [s.pixel_array for s in slices]
    image_3d = np.stack(pixel_arrays, axis=-1)
    # Converter para imagem ITK
    itk_image = sitk.GetImageFromArray(image_3d)
    return itk_image

def show_slices(image_array, num_slices=165, cols=11):
    # Calcular número de linhas necessárias
    rows = num_slices // cols + (num_slices % cols > 0)
    fig, axes = plt.subplots(rows, cols, figsize=(20, 20))

    for i in range(num_slices):
        row = i // cols
        col = i % cols
        axes[row, col].imshow(image_array[:, :, i], cmap='gray')
        axes[row, col].axis('off')
        axes[row, col].set_title(f'Slice {i}')

    # Remover subplots extras se o número de fatias for menor que o grid total
    for j in range(num_slices, rows * cols):
        fig.delaxes(axes.flatten()[j])

    plt.tight_layout()
    plt.show()

def correct_orientation(itk_image):
    # Obter a matriz de transformação atual
    origin = itk_image.GetOrigin()
    direction = itk_image.GetDirection()
    spacing = itk_image.GetSpacing()
    
    # Alterar a direção da imagem para que os eixos estejam orientados corretamente
    # Isso depende da orientação específica dos dados DICOM
    direction_corrected = [1, 0, 0, 0, 1, 0, 0, 0, -1]  # Exemplo para inverter o eixo Z
    
    itk_image.SetDirection(direction_corrected)
    
    # Atualizar a origem da imagem se necessário
    # Exemplo: modificar a origem para compensar a mudança na orientação
    origin_corrected = (origin[0], origin[1], origin[2] + (len(slices) - 1) * spacing[2])
    itk_image.SetOrigin(origin_corrected)
    
    return itk_image

# Caminho p DICOM
dicom_directory = "C:/Users/bruno/Desktop/IANS/mri_exemplo/DICOM/I60175"

# Carregar  DICOM
slices = load_dicom_images(dicom_directory, max_slices=166)  # Limitar a 166 imagens
image_3d = create_3d_image(slices)

# Corrigir a orientação da imagem
image_3d_corrected = correct_orientation(image_3d)

# Converter a imagem segmentada para um array NumPy para visualização
img_array = sitk.GetArrayFromImage(image_3d_corrected)

# Visualizar os cortes 2D da imagem segmentada (somente o cérebro)
show_slices(img_array, num_slices=166, cols=11)

# Salvar a imagem 3D segmentada como um arquivo NIfTI
sitk.WriteImage(image_3d, "C:/Users/bruno/Desktop/IANS/mri_exemplo/NIFTL/output_brain_2_oriented.nii.gz")

In [ ]:
def print_image(image):

    # Converte a imagem para um numpy array
    image_array = image.numpy()

    # Obtém o índice central para cada dimensão
    axial_index = image_array.shape[0] // 2  # Corte axial
    coronal_index = image_array.shape[1] // 2  # Corte coronal
    sagital_index = image_array.shape[2] // 2  # Corte sagital

    # Cria um gráfico com 3 subgráficos
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Exibe o corte axial
    axes[0].imshow(image_array[:, :, axial_index], cmap='gray')
    axes[0].set_title('Corte Axial')
    axes[0].axis('off')

    # Exibe o corte coronal
    axes[1].imshow(image_array[:, coronal_index, :], cmap='gray')
    axes[1].set_title('Corte Coronal')
    axes[1].axis('off')

    # Exibe o corte sagital
    axes[2].imshow(image_array[sagital_index, :, :], cmap='gray')
    axes[2].set_title('Corte Sagital')
    axes[2].axis('off')

    # Ajusta o layout e exibe os cortes
    plt.tight_layout()
    plt.show()